# 多层感知机从零开始实现

现在我们尝试自己实现一个多层感知机，依然使用Fashion-MNIST图像分类数据集。

# 加载数据集


In [12]:
import torch
import torchvision
from torch.utils import data
from torchvision import transforms
import warnings
warnings.filterwarnings('ignore')

In [16]:
trans = transforms.ToTensor()
batch_size = 256
train_data = torchvision.datasets.FashionMNIST(root='../data', train=True, download=True, transform=trans)
test_data = torchvision.datasets.FashionMNIST(root='../data', train=False, download=True, transform=trans)
train_iter = data.DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_iter = data.DataLoader(test_data, shuffle=True, batch_size=batch_size)

## 初始化模型参数
Fashion-mnist数据集的每个图像是由784个像素值组成，因此输入特征的维度为784，输出类别是10，因此输出维度为10。我们假设实现一个具有单隐藏层的多层感知机，它包含256个隐藏单元。注意，我们可以将这两个变量都视为超参数。
通常，我们选择2的若干次幂作为层的宽度。
因为内存在硬件中的分配和寻址方式，这么做往往可以在计算上更高效。


我们用几个张量来表示我们的参数。
注意，对于每一层我们都要记录一个权重矩阵和一个偏置向量。
跟以前一样，我们要为损失关于这些参数的梯度分配内存。

In [19]:
num_inputs, num_outputs, num_hiddens = 784, 10, 256
W1 = torch.normal(0, 0.01, size=(num_inputs, num_hiddens), requires_grad=True)
b1 = torch.zeros(num_hiddens, requires_grad=True)
W2 = torch.normal(0, 0.01, size=(num_hiddens, num_outputs), requires_grad=True)
b2 = torch.zeros(num_outputs, requires_grad=True)
params = [W1, b1, W2, b2]

## 激活函数

为了确保我们对模型的细节了如指掌，
我们将[**实现ReLU激活函数**]，
而不是直接调用内置的`relu`函数。

In [26]:
def relu(X):
    a = torch.zeros_like(X)
    return torch.max(X, a)

## 模型

In [28]:
def net(X):
    X = X.reshape(-1, num_inputs)
    H = relu(torch.matmul(X, W1) + b1)
    return torch.matmul(H, W2) + b2

## 损失函数
这里直接使用高级API的内置函数来计算softmax和交叉熵损失函数,需要注意torch.nn.CrossEntropy()同时完成了对输出y_hat进行指数运算、softmax运算得到概率、取出概率求交叉熵损失的操作。

另外需要注意reduction参数的含义。

In [43]:
# loss = torch.nn.CrossEntropyLoss()
loss = torch.nn.CrossEntropyLoss(reduction='none')

## 优化器

In [39]:
lr = 0.01
optimizer = torch.optim.SGD(params, lr=lr)

## 训练


In [44]:
num_epochs = 10
optimizer = torch.optim.SGD(params, lr=lr)
for epoch in range(num_epochs):
    loss_total = 0
    samples = 0
    for X, y in train_iter:
        y_hat = net(X)
        loss_batch = loss(y_hat, y)
        optimizer.zero_grad()
        loss_batch.mean().backward()
        optimizer.step()
        loss_total += loss_batch.sum()
        samples += len(y)
    print('epoch: ', epoch + 1, 'loss', loss_total / samples)

epoch:  1 loss tensor(0.4153, grad_fn=<DivBackward0>)
epoch:  2 loss tensor(0.4142, grad_fn=<DivBackward0>)
epoch:  3 loss tensor(0.4127, grad_fn=<DivBackward0>)
epoch:  4 loss tensor(0.4115, grad_fn=<DivBackward0>)
epoch:  5 loss tensor(0.4103, grad_fn=<DivBackward0>)
epoch:  6 loss tensor(0.4093, grad_fn=<DivBackward0>)
epoch:  7 loss tensor(0.4079, grad_fn=<DivBackward0>)
epoch:  8 loss tensor(0.4065, grad_fn=<DivBackward0>)
epoch:  9 loss tensor(0.4057, grad_fn=<DivBackward0>)
epoch:  10 loss tensor(0.4045, grad_fn=<DivBackward0>)


In [41]:
num_epochs =10
for epoch in range(num_epochs):
    training_loss = 0
    num_batches = 0
    for X, y in train_iter:
        y_hat = net(X)
        loss_batch = loss(y_hat, y)  # 计算的是一个batch的平均损失
        optimizer.zero_grad()
        loss_batch.backward()
        optimizer.step()
        training_loss += loss_batch  # 对每一个batch的平均损失求和
        num_batches += 1  # 记录一共有多少个batch
    print(f'loss in epoch {epoch + 1}: {training_loss / num_batches}')

loss in epoch 1: 0.42980316281318665
loss in epoch 2: 0.4278811812400818
loss in epoch 3: 0.42638543248176575
loss in epoch 4: 0.424741268157959
loss in epoch 5: 0.4237530827522278
loss in epoch 6: 0.4221717119216919
loss in epoch 7: 0.42112651467323303
loss in epoch 8: 0.4197741746902466
loss in epoch 9: 0.4182150363922119
loss in epoch 10: 0.4164640009403229
